In [1]:
import sys
sys.path.append('..')

import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd, numpy as np, gc
from tqdm import tqdm
import joblib
import dask.multiprocessing
import cudf, cupy, time
from numba import jit, njit, prange
from sklearn.metrics import precision_recall_curve, auc, log_loss
# from sklearn.model_selection import train_test_split
from dask_ml.model_selection import train_test_split
import xgboost as xgb

dask.config.set(schedular='process')
pd.set_option('display.max_columns', 500)

from utils.cuda_cluster import *
from utils.preprocessing import read_data, factorize_small_cardinality
from utils.util import *
from utils.evaluate import calculate_ctr, compute_rce, average_precision_score


import core.config as conf

/home/hyez/anaconda3/envs/dask-cudf/lib/python3.7/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 35725 instead
  http_address["port"], self.http_server.port


In [2]:
path = f'{conf.preproc_path}/train/part-00050.parquet'
valid = read_data(path)
gc.collect()
save_memory( valid )

In [3]:
label_names = ['reply', 'retweet', 'retweet_comment', 'like']
DONT_USE = ['timestamp','creator_account_creation','engager_account_creation','engage_time',
            'creator_account_creation', 'engager_account_creation',
            'fold','tweet_id', 
            'tr','dt_day','','',
            'engager_id','creator_id','engager_is_verified',
            'elapsed_time',
            'links','domains','hashtags0','hashtags1',
            'hashtags','tweet_hash','dt_second','id',
            'tw_hash0',
            'tw_hash1',
            'tw_rt_uhash',
            'same_language', 'nan_language','language',
            'tw_hash', 'tw_freq_hash','tw_first_word', 'tw_second_word', 'tw_last_word', 'tw_llast_word',
            'ypred','creator_count_combined','creator_user_fer_count_delta_time','creator_user_fing_count_delta_time','creator_user_fering_count_delta_time','creator_user_fing_count_mode','creator_user_fer_count_mode','creator_user_fering_count_mode'
            
           ]
DONT_USE += label_names
RMV = [c for c in DONT_USE if c in valid.columns]

print('Using %i features:'%(len(RMV)))
np.asarray(RMV)

Using 9 features:


array(['tweet_id', 'dt_day', 'engager_id', 'creator_id', 'domains',
       'language', 'reply', 'retweet', 'like'], dtype='<U10')

In [4]:
TARGET = 'like'
model = joblib.load( f'/hdd/models/model-'+TARGET+'-49.xgb' )
dvalid = xgb.DMatrix(data=valid.drop(RMV, axis=1).compute().to_pandas() ,label=valid[TARGET].compute().values)
valid = valid.compute()
valid['ypred'] = model.predict(dvalid)
del dvalid, model
_=gc.collect()

In [5]:
valid[['like', 'ypred']]

,like,ypred
id,,
1,0,0.481552
2,1,0.667589
3,0,0.806838
4,0,0.341133
5,0,0.046326
...,...,...
3034000,0,0.017058
3034001,0,0.560991
3034002,0,0.079748


In [6]:
rce_like = compute_rce(valid['ypred'].to_array(), valid['like'].to_array())
rce_like

10.02013208018122

In [8]:
pred = valid['ypred'].to_array()
pred = list(map(lambda x: 1 if x > 0.4 else 0,  pred))
valid['ypred'] = pred
ap_like = average_precision_score(valid['ypred'].to_array(), valid['like'].to_array())
ap_like

0.4395012912796188